In [1]:
%matplotlib inline


How to reproduce the doc2vec 'Paragraph Vector' paper
=====================================================

Shows how to reproduce results of the "Distributed Representation of Sentences and Documents" paper by Le and Mikolov using Gensim.




In [2]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

Introduction
------------

This guide shows you how to reproduce the results of the paper by `Le and
Mikolov 2014 <https://arxiv.org/pdf/1405.4053.pdf>`_ using Gensim. While the
entire paper is worth reading (it's only 9 pages), we will be focusing on
Section 3.2: "Beyond One Sentence - Sentiment Analysis with the IMDB
dataset".

This guide follows the following steps:

#. Load the IMDB dataset
#. Train a variety of Doc2Vec models on the dataset
#. Evaluate the performance of each model using a logistic regression
#. Examine some of the results directly:

When examining results, we will look for answers for the following questions:

#. Are inferred vectors close to the precalculated ones?
#. Do close documents seem more related than distant ones?
#. Do the word vectors show useful similarities?
#. Are the word vectors from this dataset any good at analogies?

Load corpus
-----------

Our data for the tutorial will be the `IMDB archive
<http://ai.stanford.edu/~amaas/data/sentiment/>`_.
If you're not familiar with this dataset, then here's a brief intro: it
contains several thousand movie reviews.

Each review is a single line of text containing multiple sentences, for example:

```
One of the best movie-dramas I have ever seen. We do a lot of acting in the
church and this is one that can be used as a resource that highlights all the
good things that actors can do in their work. I highly recommend this one,
especially for those who have an interest in acting, as a "must see."
```

These reviews will be the **documents** that we will work with in this tutorial.
There are 100 thousand reviews in total.

#. 25k reviews for training (12.5k positive, 12.5k negative)
#. 25k reviews for testing (12.5k positive, 12.5k negative)
#. 50k unlabeled reviews

Out of 100k reviews, 50k have a label: either positive (the reviewer liked
the movie) or negative.
The remaining 50k are unlabeled.

Our first task will be to prepare the dataset.

More specifically, we will:

#. Download the tar.gz file (it's only 84MB, so this shouldn't take too long)
#. Unpack it and extract each movie review
#. Split the reviews into training and test datasets

First, let's define a convenient datatype for holding data for a single document:

* words: The text of the document, as a ``list`` of words.
* tags: Used to keep the index of the document in the entire dataset.
* split: one of ``train``\ , ``test`` or ``extra``. Determines how the document will be used (for training, testing, etc).
* sentiment: either 1 (positive), 0 (negative) or None (unlabeled document).

This data type is helpful for later evaluation and reporting.
In particular, the ``index`` member will help us quickly and easily retrieve the vectors for a document from a model.




In [3]:
import collections

SentimentDocument = collections.namedtuple('SentimentDocument', 'words tags split sentiment')

We can now proceed with loading the corpus.



In [8]:
import io
import re
import tarfile
import os.path

import smart_open
import gensim.utils

def download_dataset(url='http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'):
    fname = url.split('/')[-1]

    if os.path.isfile(fname):
       return fname

    # Download the file to local storage first.
    with smart_open.open(url, "rb", ignore_ext=True) as fin:
        with smart_open.open(fname, 'wb', ignore_ext=True) as fout:
            while True:
                buf = fin.read(io.DEFAULT_BUFFER_SIZE)
                if not buf:
                    break
                fout.write(buf)

    return fname

def create_sentiment_document(name, text, index):
    _, split, sentiment_str, _ = name.split('/')
    sentiment = {'pos': 1.0, 'neg': 0.0, 'unsup': None}[sentiment_str]

    if sentiment is None:
        split = 'extra'

    tokens = gensim.utils.to_unicode(text).split()
    return SentimentDocument(tokens, [index], split, sentiment)

def extract_documents():
    fname = download_dataset()

    index = 0

    with tarfile.open(fname, mode='r:gz') as tar:
        for member in tar.getmembers():
            if re.match(r'aclImdb/(train|test)/(pos|neg|unsup)/\d+_\d+.txt$', member.name):
                member_bytes = tar.extractfile(member).read()
                member_text = member_bytes.decode('utf-8', errors='replace')
                assert member_text.count('\n') == 0
                yield create_sentiment_document(member.name, member_text, index)
                index += 1

alldocs = list(extract_documents())

Here's what a single document looks like.



In [9]:
print(alldocs[27])

SentimentDocument(words=['I', 'was', 'looking', 'forward', 'to', 'this', 'movie.', 'Trustworthy', 'actors,', 'interesting', 'plot.', 'Great', 'atmosphere', 'then', '?????', 'IF', 'you', 'are', 'going', 'to', 'attempt', 'something', 'that', 'is', 'meant', 'to', 'encapsulate', 'the', 'meaning', 'of', 'life.', 'First.', 'Know', 'it.', 'OK', 'I', 'did', 'not', 'expect', 'the', 'directors', 'or', 'writers', 'to', 'actually', 'know', 'the', 'meaning', 'but', 'I', 'thought', 'they', 'may', 'have', 'offered', 'crumbs', 'to', 'peck', 'at', 'and', 'treats', 'to', 'add', 'fuel', 'to', 'the', 'fire-Which!', 'they', 'almost', 'did.', 'Things', 'I', "didn't", 'get.', 'A', 'woman', 'wandering', 'around', 'in', 'dark', 'places', 'and', 'lonely', 'car', 'parks', 'alone-oblivious', 'to', 'the', 'consequences.', 'Great', 'riddles', 'that', 'fell', 'by', 'the', 'wayside.', 'The', 'promise', 'of', 'the', 'knowledge', 'therein', 'contained', 'by', 'the', 'original', 'so-called', 'criminal.', 'I', 'had', 'no

Extract our documents and split into training/test sets.



In [10]:
train_docs = [doc for doc in alldocs if doc.split == 'train']
test_docs = [doc for doc in alldocs if doc.split == 'test']
print(f'{len(alldocs)} docs: {len(train_docs)} train-sentiment, {len(test_docs)} test-sentiment')

100000 docs: 25000 train-sentiment, 25000 test-sentiment


Set-up Doc2Vec Training & Evaluation Models
-------------------------------------------

We approximate the experiment of Le & Mikolov `"Distributed Representations
of Sentences and Documents"
<http://cs.stanford.edu/~quocle/paragraph_vector.pdf>`_ with guidance from
Mikolov's `example go.sh
<https://groups.google.com/d/msg/word2vec-toolkit/Q49FIrNOQRo/J6KG8mUj45sJ>`_::

    ./word2vec -train ../alldata-id.txt -output vectors.txt -cbow 0 -size 100 -window 10 -negative 5 -hs 0 -sample 1e-4 -threads 40 -binary 0 -iter 20 -min-count 1 -sentence-vectors 1

We vary the following parameter choices:

* 100-dimensional vectors, as the 400-d vectors of the paper take a lot of
  memory and, in our tests of this task, don't seem to offer much benefit
* Similarly, frequent word subsampling seems to decrease sentiment-prediction
  accuracy, so it's left out
* ``cbow=0`` means skip-gram which is equivalent to the paper's 'PV-DBOW'
  mode, matched in gensim with ``dm=0``
* Added to that DBOW model are two DM models, one which averages context
  vectors (\ ``dm_mean``\ ) and one which concatenates them (\ ``dm_concat``\ ,
  resulting in a much larger, slower, more data-hungry model)
* A ``min_count=2`` saves quite a bit of model memory, discarding only words
  that appear in a single doc (and are thus no more expressive than the
  unique-to-each doc vectors themselves)




In [11]:
import multiprocessing
from collections import OrderedDict

import gensim.models.doc2vec
assert gensim.models.doc2vec.FAST_VERSION > -1, "This will be painfully slow otherwise"

from gensim.models.doc2vec import Doc2Vec

common_kwargs = dict(
    vector_size=100, epochs=20, min_count=2,
    sample=0, workers=multiprocessing.cpu_count(), negative=5, hs=0,
)

simple_models = [
    # PV-DBOW plain
    Doc2Vec(dm=0, **common_kwargs),
    # PV-DM w/ default averaging; a higher starting alpha may improve CBOW/PV-DM modes
    Doc2Vec(dm=1, window=10, alpha=0.05, comment='alpha=0.05', **common_kwargs),
    # PV-DM w/ concatenation - big, slow, experimental mode
    # window=5 (both sides) approximates paper's apparent 10-word total window size
    Doc2Vec(dm=1, dm_concat=1, window=5, **common_kwargs),
]

for model in simple_models:
    model.build_vocab(alldocs)
    print(f"{model} vocabulary scanned & state initialized")

models_by_name = OrderedDict((str(model), model) for model in simple_models)

2021-10-30 18:07:47,073 : INFO : Doc2Vec lifecycle event {'params': 'Doc2Vec(dbow,d100,n5,mc2,t4)', 'datetime': '2021-10-30T18:07:47.073629', 'gensim': '4.1.2', 'python': '3.8.8 (default, Apr 13 2021, 19:58:26) \n[GCC 7.3.0]', 'platform': 'Linux-5.4.0-89-generic-x86_64-with-glibc2.10', 'event': 'created'}
2021-10-30 18:07:47,079 : INFO : Doc2Vec lifecycle event {'params': 'Doc2Vec(dm/m,d100,n5,w10,mc2,t4)', 'datetime': '2021-10-30T18:07:47.079165', 'gensim': '4.1.2', 'python': '3.8.8 (default, Apr 13 2021, 19:58:26) \n[GCC 7.3.0]', 'platform': 'Linux-5.4.0-89-generic-x86_64-with-glibc2.10', 'event': 'created'}
2021-10-30 18:07:47,080 : INFO : using concatenative 1100-dimensional layer1
2021-10-30 18:07:47,082 : INFO : Doc2Vec lifecycle event {'params': 'Doc2Vec(dm/c,d100,n5,w5,mc2,t4)', 'datetime': '2021-10-30T18:07:47.082242', 'gensim': '4.1.2', 'python': '3.8.8 (default, Apr 13 2021, 19:58:26) \n[GCC 7.3.0]', 'platform': 'Linux-5.4.0-89-generic-x86_64-with-glibc2.10', 'event': 'creat

Doc2Vec(dbow,d100,n5,mc2,t4) vocabulary scanned & state initialized


2021-10-30 18:07:59,866 : INFO : PROGRESS: at example #10000, processed 2292381 words (4256245/s), 150816 word types, 0 tags
2021-10-30 18:08:00,385 : INFO : PROGRESS: at example #20000, processed 4573645 words (4408284/s), 238497 word types, 0 tags
2021-10-30 18:08:00,937 : INFO : PROGRESS: at example #30000, processed 6865575 words (4158206/s), 312348 word types, 0 tags
2021-10-30 18:08:01,500 : INFO : PROGRESS: at example #40000, processed 9190019 words (4130910/s), 377231 word types, 0 tags
2021-10-30 18:08:02,093 : INFO : PROGRESS: at example #50000, processed 11557847 words (3998898/s), 438729 word types, 0 tags
2021-10-30 18:08:02,717 : INFO : PROGRESS: at example #60000, processed 13899883 words (3758081/s), 493913 word types, 0 tags
2021-10-30 18:08:03,323 : INFO : PROGRESS: at example #70000, processed 16270094 words (3920051/s), 548474 word types, 0 tags
2021-10-30 18:08:03,902 : INFO : PROGRESS: at example #80000, processed 18598876 words (4021380/s), 598272 word types, 0 t

Doc2Vec(dm/m,d100,n5,w10,mc2,t4) vocabulary scanned & state initialized


2021-10-30 18:08:11,258 : INFO : PROGRESS: at example #10000, processed 2292381 words (4890275/s), 150816 word types, 0 tags
2021-10-30 18:08:11,759 : INFO : PROGRESS: at example #20000, processed 4573645 words (4561823/s), 238497 word types, 0 tags
2021-10-30 18:08:12,379 : INFO : PROGRESS: at example #30000, processed 6865575 words (3703002/s), 312348 word types, 0 tags
2021-10-30 18:08:12,938 : INFO : PROGRESS: at example #40000, processed 9190019 words (4161006/s), 377231 word types, 0 tags
2021-10-30 18:08:13,441 : INFO : PROGRESS: at example #50000, processed 11557847 words (4713860/s), 438729 word types, 0 tags
2021-10-30 18:08:13,941 : INFO : PROGRESS: at example #60000, processed 13899883 words (4684956/s), 493913 word types, 0 tags
2021-10-30 18:08:14,614 : INFO : PROGRESS: at example #70000, processed 16270094 words (3524436/s), 548474 word types, 0 tags
2021-10-30 18:08:15,205 : INFO : PROGRESS: at example #80000, processed 18598876 words (3946265/s), 598272 word types, 0 t

Doc2Vec(dm/c,d100,n5,w5,mc2,t4) vocabulary scanned & state initialized


Le and Mikolov note that combining a paragraph vector from Distributed Bag of
Words (DBOW) and Distributed Memory (DM) improves performance. We will
follow, pairing the models together for evaluation. Here, we concatenate the
paragraph vectors obtained from each model with the help of a thin wrapper
class included in a gensim test module. (Note that this a separate, later
concatenation of output-vectors than the kind of input-window-concatenation
enabled by the ``dm_concat=1`` mode above.)




In [14]:
from gensim.test.test_doc2vec import ConcatenatedDoc2Vec
models_by_name['dbow+dmm'] = ConcatenatedDoc2Vec([simple_models[0], simple_models[1]])
models_by_name['dbow+dmc'] = ConcatenatedDoc2Vec([simple_models[0], simple_models[2]])

2021-10-30 18:10:30,520 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2021-10-30 18:10:30,522 : INFO : built Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...) from 9 documents (total 29 corpus positions)
2021-10-30 18:10:30,524 : INFO : Dictionary lifecycle event {'msg': "built Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...) from 9 documents (total 29 corpus positions)", 'datetime': '2021-10-30T18:10:30.524356', 'gensim': '4.1.2', 'python': '3.8.8 (default, Apr 13 2021, 19:58:26) \n[GCC 7.3.0]', 'platform': 'Linux-5.4.0-89-generic-x86_64-with-glibc2.10', 'event': 'created'}


Predictive Evaluation Methods
-----------------------------

Given a document, our ``Doc2Vec`` models output a vector representation of the document.
How useful is a particular model?
In case of sentiment analysis, we want the ouput vector to reflect the sentiment in the input document.
So, in vector space, positive documents should be distant from negative documents.

We train a logistic regression from the training set:

  - regressors (inputs): document vectors from the Doc2Vec model
  - target (outpus): sentiment labels

So, this logistic regression will be able to predict sentiment given a document vector.

Next, we test our logistic regression on the test set, and measure the rate of errors (incorrect predictions).
If the document vectors from the Doc2Vec model reflect the actual sentiment well, the error rate will be low.

Therefore, the error rate of the logistic regression is indication of *how well* the given Doc2Vec model represents documents as vectors.
We can then compare different ``Doc2Vec`` models by looking at their error rates.




In [15]:
import numpy as np
import statsmodels.api as sm
from random import sample

def logistic_predictor_from_data(train_targets, train_regressors):
    """Fit a statsmodel logistic predictor on supplied data"""
    logit = sm.Logit(train_targets, train_regressors)
    predictor = logit.fit(disp=0)
    # print(predictor.summary())
    return predictor

def error_rate_for_model(test_model, train_set, test_set):
    """Report error rate on test_doc sentiments, using supplied model and train_docs"""

    train_targets = [doc.sentiment for doc in train_set]
    train_regressors = [test_model.dv[doc.tags[0]] for doc in train_set]
    train_regressors = sm.add_constant(train_regressors)
    predictor = logistic_predictor_from_data(train_targets, train_regressors)

    test_regressors = [test_model.dv[doc.tags[0]] for doc in test_set]
    test_regressors = sm.add_constant(test_regressors)

    # Predict & evaluate
    test_predictions = predictor.predict(test_regressors)
    corrects = sum(np.rint(test_predictions) == [doc.sentiment for doc in test_set])
    errors = len(test_predictions) - corrects
    error_rate = float(errors) / len(test_predictions)
    return (error_rate, errors, len(test_predictions), predictor)

2021-10-30 18:11:07,264 : INFO : NumExpr defaulting to 4 threads.


Bulk Training & Per-Model Evaluation
------------------------------------

Note that doc-vector training is occurring on *all* documents of the dataset,
which includes all TRAIN/TEST/DEV docs.  Because the native document-order
has similar-sentiment documents in large clumps – which is suboptimal for
training – we work with once-shuffled copy of the training set.

We evaluate each model's sentiment predictive power based on error rate, and
the evaluation is done for each model.

(On a 4-core 2.6Ghz Intel Core i7, these 20 passes training and evaluating 3
main models takes about an hour.)




In [16]:
from collections import defaultdict
error_rates = defaultdict(lambda: 1.0)  # To selectively print only best errors achieved

In [17]:
from random import shuffle
shuffled_alldocs = alldocs[:]
shuffle(shuffled_alldocs)

for model in simple_models:
    print(f"Training {model}")
    model.train(shuffled_alldocs, total_examples=len(shuffled_alldocs), epochs=model.epochs)

    print(f"\nEvaluating {model}")
    err_rate, err_count, test_count, predictor = error_rate_for_model(model, train_docs, test_docs)
    error_rates[str(model)] = err_rate
    print("\n%f %s\n" % (err_rate, model))

for model in [models_by_name['dbow+dmm'], models_by_name['dbow+dmc']]:
    print(f"\nEvaluating {model}")
    err_rate, err_count, test_count, predictor = error_rate_for_model(model, train_docs, test_docs)
    error_rates[str(model)] = err_rate
    print(f"\n{err_rate} {model}\n")

2021-10-30 18:11:20,054 : INFO : Doc2Vec lifecycle event {'msg': 'training model with 4 workers on 265408 vocabulary and 100 features, using sg=1 hs=0 sample=0 negative=5 window=5 shrink_windows=True', 'datetime': '2021-10-30T18:11:20.054018', 'gensim': '4.1.2', 'python': '3.8.8 (default, Apr 13 2021, 19:58:26) \n[GCC 7.3.0]', 'platform': 'Linux-5.4.0-89-generic-x86_64-with-glibc2.10', 'event': 'train'}


Training Doc2Vec(dbow,d100,n5,mc2,t4)


2021-10-30 18:11:21,066 : INFO : EPOCH 1 - PROGRESS: at 2.50% examples, 568413 words/s, in_qsize 8, out_qsize 0
2021-10-30 18:11:22,073 : INFO : EPOCH 1 - PROGRESS: at 5.07% examples, 576538 words/s, in_qsize 7, out_qsize 0
2021-10-30 18:11:23,077 : INFO : EPOCH 1 - PROGRESS: at 7.96% examples, 609452 words/s, in_qsize 7, out_qsize 0
2021-10-30 18:11:24,080 : INFO : EPOCH 1 - PROGRESS: at 11.47% examples, 659856 words/s, in_qsize 7, out_qsize 0
2021-10-30 18:11:25,080 : INFO : EPOCH 1 - PROGRESS: at 14.85% examples, 684331 words/s, in_qsize 7, out_qsize 0
2021-10-30 18:11:26,108 : INFO : EPOCH 1 - PROGRESS: at 18.35% examples, 700848 words/s, in_qsize 7, out_qsize 0
2021-10-30 18:11:27,132 : INFO : EPOCH 1 - PROGRESS: at 21.05% examples, 685196 words/s, in_qsize 7, out_qsize 0
2021-10-30 18:11:28,140 : INFO : EPOCH 1 - PROGRESS: at 23.87% examples, 680175 words/s, in_qsize 7, out_qsize 0
2021-10-30 18:11:29,143 : INFO : EPOCH 1 - PROGRESS: at 27.89% examples, 706417 words/s, in_qsize 7


Evaluating Doc2Vec(dbow,d100,n5,mc2,t4)


2021-10-30 18:21:23,913 : INFO : Doc2Vec lifecycle event {'msg': 'training model with 4 workers on 265408 vocabulary and 100 features, using sg=0 hs=0 sample=0 negative=5 window=10 shrink_windows=True', 'datetime': '2021-10-30T18:21:23.913160', 'gensim': '4.1.2', 'python': '3.8.8 (default, Apr 13 2021, 19:58:26) \n[GCC 7.3.0]', 'platform': 'Linux-5.4.0-89-generic-x86_64-with-glibc2.10', 'event': 'train'}


...


Evaluating Doc2Vec(dm/m,d100,n5,w10,mc2,t4)


2021-10-30 18:37:21,724 : INFO : Doc2Vec lifecycle event {'msg': 'training model with 4 workers on 265409 vocabulary and 1100 features, using sg=0 hs=0 sample=0 negative=5 window=5 shrink_windows=True', 'datetime': '2021-10-30T18:37:21.724261', 'gensim': '4.1.2', 'python': '3.8.8 (default, Apr 13 2021, 19:58:26) \n[GCC 7.3.0]', 'platform': 'Linux-5.4.0-89-generic-x86_64-with-glibc2.10', 'event': 'train'}



0.170760 Doc2Vec(dm/m,d100,n5,w10,mc2,t4)

Training Doc2Vec(dm/c,d100,n5,w5,mc2,t4)


2021-10-30 18:37:22,911 : INFO : EPOCH 1 - PROGRESS: at 0.65% examples, 122847 words/s, in_qsize 7, out_qsize 0
2021-10-30 18:37:24,098 : INFO : EPOCH 1 - PROGRESS: at 0.92% examples, 85826 words/s, in_qsize 7, out_qsize 0
2021-10-30 18:37:25,102 : INFO : EPOCH 1 - PROGRESS: at 2.35% examples, 157625 words/s, in_qsize 7, out_qsize 0
2021-10-30 18:37:26,130 : INFO : EPOCH 1 - PROGRESS: at 3.94% examples, 204347 words/s, in_qsize 7, out_qsize 0
2021-10-30 18:37:27,160 : INFO : EPOCH 1 - PROGRESS: at 5.48% examples, 231352 words/s, in_qsize 7, out_qsize 0
2021-10-30 18:37:28,167 : INFO : EPOCH 1 - PROGRESS: at 7.10% examples, 253554 words/s, in_qsize 7, out_qsize 0
2021-10-30 18:37:29,182 : INFO : EPOCH 1 - PROGRESS: at 8.66% examples, 267394 words/s, in_qsize 7, out_qsize 0
2021-10-30 18:37:30,247 : INFO : EPOCH 1 - PROGRESS: at 10.24% examples, 277262 words/s, in_qsize 7, out_qsize 0
2021-10-30 18:37:31,251 : INFO : EPOCH 1 - PROGRESS: at 11.85% examples, 287523 words/s, in_qsize 7, out


Evaluating Doc2Vec(dm/c,d100,n5,w5,mc2,t4)

0.303200 Doc2Vec(dm/c,d100,n5,w5,mc2,t4)


Evaluating Doc2Vec(dbow,d100,n5,mc2,t4)+Doc2Vec(dm/m,d100,n5,w10,mc2,t4)

0.1046 Doc2Vec(dbow,d100,n5,mc2,t4)+Doc2Vec(dm/m,d100,n5,w10,mc2,t4)


Evaluating Doc2Vec(dbow,d100,n5,mc2,t4)+Doc2Vec(dm/c,d100,n5,w5,mc2,t4)

0.10484 Doc2Vec(dbow,d100,n5,mc2,t4)+Doc2Vec(dm/c,d100,n5,w5,mc2,t4)



Achieved Sentiment-Prediction Accuracy
--------------------------------------
Compare error rates achieved, best-to-worst



In [18]:
print("Err_rate Model")
for rate, name in sorted((rate, name) for name, rate in error_rates.items()):
    print(f"{rate} {name}")

Err_rate Model
0.1046 Doc2Vec(dbow,d100,n5,mc2,t4)+Doc2Vec(dm/m,d100,n5,w10,mc2,t4)
0.10484 Doc2Vec(dbow,d100,n5,mc2,t4)+Doc2Vec(dm/c,d100,n5,w5,mc2,t4)
0.10636 Doc2Vec(dbow,d100,n5,mc2,t4)
0.17076 Doc2Vec(dm/m,d100,n5,w10,mc2,t4)
0.3032 Doc2Vec(dm/c,d100,n5,w5,mc2,t4)


In our testing, contrary to the results of the paper, on this problem,
PV-DBOW alone performs as good as anything else. Concatenating vectors from
different models only sometimes offers a tiny predictive improvement – and
stays generally close to the best-performing solo model included.

The best results achieved here are just around 10% error rate, still a long
way from the paper's reported 7.42% error rate.

(Other trials not shown, with larger vectors and other changes, also don't
come close to the paper's reported value. Others around the net have reported
a similar inability to reproduce the paper's best numbers. The PV-DM/C mode
improves a bit with many more training epochs – but doesn't reach parity with
PV-DBOW.)




Examining Results
-----------------

Let's look for answers to the following questions:

#. Are inferred vectors close to the precalculated ones?
#. Do close documents seem more related than distant ones?
#. Do the word vectors show useful similarities?
#. Are the word vectors from this dataset any good at analogies?




Are inferred vectors close to the precalculated ones?
-----------------------------------------------------



In [19]:
doc_id = np.random.randint(len(simple_models[0].dv))  # Pick random doc; re-run cell for more examples
print(f'for doc {doc_id}...')
for model in simple_models:
    inferred_docvec = model.infer_vector(alldocs[doc_id].words)
    print(f'{model}:\n {model.dv.most_similar([inferred_docvec], topn=3)}')

for doc 20802...
Doc2Vec(dbow,d100,n5,mc2,t4):
 [(20802, 0.9696964025497437), (64261, 0.6012817621231079), (49615, 0.5952827334403992)]
Doc2Vec(dm/m,d100,n5,w10,mc2,t4):
 [(20802, 0.9264684915542603), (73690, 0.5946333408355713), (7601, 0.5910226702690125)]
Doc2Vec(dm/c,d100,n5,w5,mc2,t4):
 [(20802, 0.7576104998588562), (94849, 0.43759655952453613), (41635, 0.4262971580028534)]


(Yes, here the stored vector from 20 epochs of training is usually one of the
closest to a freshly-inferred vector for the same words. Defaults for
inference may benefit from tuning for each dataset or model parameters.)




Do close documents seem more related than distant ones?
-------------------------------------------------------



In [20]:
import random

doc_id = np.random.randint(len(simple_models[0].dv))  # pick random doc, re-run cell for more examples
model = random.choice(simple_models)  # and a random model
sims = model.dv.most_similar(doc_id, topn=len(model.dv))  # get *all* similar documents
print(f'TARGET ({doc_id}): «{" ".join(alldocs[doc_id].words)}»\n')
print(f'SIMILAR/DISSIMILAR DOCS PER MODEL {model}%s:\n')
for label, index in [('MOST', 0), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    s = sims[index]
    i = sims[index][0]
    words = ' '.join(alldocs[i].words)
    print(f'{label} {s}: «{words}»\n')

TARGET (77978): «Indigo was delightful. It provided a wonderful story line about family values and families working together in times of trouble. It was captured in a gorgeous setting, and it educated me about these gifted Indigo children and the difficulties they face.<br /><br />It seems that the emphasis and lessons of the movie was learning how to accept one another as they are and offering love and support in an unconditional manner.<br /><br />In summary, I have recommended this film to my friends as a good one to take all the family, and I would like to see more films of this nature in the theaters as I might go more often.<br /><br />Roberta»

SIMILAR/DISSIMILAR DOCS PER MODEL Doc2Vec(dm/m,d100,n5,w10,mc2,t4)%s:

MOST (44788, 0.6669732928276062): «This film is brilliant! It touches everyone who sees it in an extraordinary way. It really takes you back to your youth and puts a new perspective on how you view your childhood memories. There are so many layers to this film. It is i

Somewhat, in terms of reviewer tone, movie genre, etc... the MOST
cosine-similar docs usually seem more like the TARGET than the MEDIAN or
LEAST... especially if the MOST has a cosine-similarity > 0.5. Re-run the
cell to try another random target document.




Do the word vectors show useful similarities?
---------------------------------------------




In [21]:
import random

word_models = simple_models[:]

def pick_random_word(model, threshold=10):
    # pick a random word with a suitable number of occurences
    while True:
        word = random.choice(model.wv.index_to_key)
        if model.wv.get_vecattr(word, "count") > threshold:
            return word

target_word = pick_random_word(word_models[0])
# or uncomment below line, to just pick a word from the relevant domain:
# target_word = 'comedy/drama'

for model in word_models:
    print(f'target_word: {repr(target_word)} model: {model} similar words:')
    for i, (word, sim) in enumerate(model.wv.most_similar(target_word, topn=10), 1):
        print(f'    {i}. {sim:.2f} {repr(word)}')
    print()

target_word: 'rails.' model: Doc2Vec(dbow,d100,n5,mc2,t4) similar words:
    1. 0.46 'realism'
    2. 0.45 'end."'
    3. 0.43 "'Janeway'"
    4. 0.42 'Sakal'
    5. 0.42 'turmoil,'
    6. 0.40 'Cappeletti'
    7. 0.40 'morbid'
    8. 0.40 'gorier'
    9. 0.40 'Esther,'
    10. 0.40 'congressmen'

target_word: 'rails.' model: Doc2Vec(dm/m,d100,n5,w10,mc2,t4) similar words:
    1. 0.65 'map.'
    2. 0.63 'stairs.'
    3. 0.63 'mark.'
    4. 0.63 'top.'
    5. 0.63 'top!'
    6. 0.62 'ground.'
    7. 0.62 'fighting!'
    8. 0.62 'ship.'
    9. 0.62 'ordinary.'
    10. 0.61 'cannon.'

target_word: 'rails.' model: Doc2Vec(dm/c,d100,n5,w5,mc2,t4) similar words:
    1. 0.69 'motions.'
    2. 0.68 'same?'
    3. 0.67 'tubes.'
    4. 0.67 'Louvre.'
    5. 0.66 'original..'
    6. 0.65 'gutter.'
    7. 0.65 'tunnels.'
    8. 0.65 'ending.)'
    9. 0.65 'clan.'
    10. 0.65 'ghettos.'



Do the DBOW words look meaningless? That's because the gensim DBOW model
doesn't train word vectors – they remain at their random initialized values –
unless you ask with the ``dbow_words=1`` initialization parameter. Concurrent
word-training slows DBOW mode significantly, and offers little improvement
(and sometimes a little worsening) of the error rate on this IMDB
sentiment-prediction task, but may be appropriate on other tasks, or if you
also need word-vectors.

Words from DM models tend to show meaningfully similar words when there are
many examples in the training data (as with 'plot' or 'actor'). (All DM modes
inherently involve word-vector training concurrent with doc-vector training.)




Are the word vectors from this dataset any good at analogies?
-------------------------------------------------------------



In [22]:
from gensim.test.utils import datapath
questions_filename = datapath('questions-words.txt')

# Note: this analysis takes many minutes
for model in word_models:
    score, sections = model.wv.evaluate_word_analogies(questions_filename)
    correct, incorrect = len(sections[-1]['correct']), len(sections[-1]['incorrect'])
    print(f'{model}: {float(correct*100)/(correct+incorrect):0.2f}%% correct ({correct} of {correct+incorrect}')

2021-10-30 19:04:51,500 : INFO : Evaluating word analogies for top 300000 words in the model on /home/sebacastillo/anaconda3/lib/python3.8/site-packages/gensim/test/test_data/questions-words.txt
2021-10-30 19:04:55,733 : INFO : capital-common-countries: 0.0% (0/420)
2021-10-30 19:05:04,560 : INFO : capital-world: 0.0% (0/902)
2021-10-30 19:05:05,392 : INFO : currency: 0.0% (0/86)
2021-10-30 19:05:19,794 : INFO : city-in-state: 0.0% (0/1510)
2021-10-30 19:05:24,588 : INFO : family: 0.0% (0/506)
2021-10-30 19:05:33,950 : INFO : gram1-adjective-to-adverb: 0.0% (0/992)
2021-10-30 19:05:41,253 : INFO : gram2-opposite: 0.0% (0/756)
2021-10-30 19:05:53,853 : INFO : gram3-comparative: 0.0% (0/1332)
2021-10-30 19:06:03,694 : INFO : gram4-superlative: 0.0% (0/1056)
2021-10-30 19:06:13,392 : INFO : gram5-present-participle: 0.0% (0/992)
2021-10-30 19:06:27,304 : INFO : gram6-nationality-adjective: 0.0% (0/1445)
2021-10-30 19:06:42,152 : INFO : gram7-past-tense: 0.0% (0/1560)
2021-10-30 19:06:53,0

Doc2Vec(dbow,d100,n5,mc2,t4): 0.00%% correct (0 of 13617


2021-10-30 19:07:04,777 : INFO : capital-common-countries: 4.3% (18/420)
2021-10-30 19:07:13,099 : INFO : capital-world: 0.7% (6/902)
2021-10-30 19:07:13,896 : INFO : currency: 0.0% (0/86)
2021-10-30 19:07:27,482 : INFO : city-in-state: 0.3% (4/1510)
2021-10-30 19:07:32,294 : INFO : family: 37.7% (191/506)
2021-10-30 19:07:40,490 : INFO : gram1-adjective-to-adverb: 4.0% (40/992)
2021-10-30 19:07:46,848 : INFO : gram2-opposite: 6.2% (47/756)
2021-10-30 19:07:59,960 : INFO : gram3-comparative: 45.0% (600/1332)
2021-10-30 19:08:10,243 : INFO : gram4-superlative: 22.1% (233/1056)
2021-10-30 19:08:19,782 : INFO : gram5-present-participle: 23.6% (234/992)
2021-10-30 19:08:32,734 : INFO : gram6-nationality-adjective: 3.7% (53/1445)
2021-10-30 19:08:46,670 : INFO : gram7-past-tense: 26.9% (420/1560)
2021-10-30 19:08:57,297 : INFO : gram8-plural: 19.2% (229/1190)
2021-10-30 19:09:04,607 : INFO : gram9-plural-verbs: 43.0% (374/870)
2021-10-30 19:09:04,608 : INFO : Quadruplets with out-of-vocabul

Doc2Vec(dm/m,d100,n5,w10,mc2,t4): 17.98%% correct (2449 of 13617


2021-10-30 19:09:08,380 : INFO : capital-common-countries: 2.6% (11/420)
2021-10-30 19:09:16,107 : INFO : capital-world: 0.9% (8/902)
2021-10-30 19:09:16,864 : INFO : currency: 0.0% (0/86)
2021-10-30 19:09:29,914 : INFO : city-in-state: 0.1% (2/1510)
2021-10-30 19:09:34,523 : INFO : family: 39.7% (201/506)
2021-10-30 19:09:43,141 : INFO : gram1-adjective-to-adverb: 7.3% (72/992)
2021-10-30 19:09:49,447 : INFO : gram2-opposite: 4.6% (35/756)
2021-10-30 19:10:00,499 : INFO : gram3-comparative: 36.6% (488/1332)
2021-10-30 19:10:09,062 : INFO : gram4-superlative: 25.6% (270/1056)
2021-10-30 19:10:17,185 : INFO : gram5-present-participle: 36.5% (362/992)
2021-10-30 19:10:28,702 : INFO : gram6-nationality-adjective: 1.4% (20/1445)
2021-10-30 19:10:43,278 : INFO : gram7-past-tense: 25.0% (390/1560)
2021-10-30 19:10:55,777 : INFO : gram8-plural: 10.3% (122/1190)
2021-10-30 19:11:04,677 : INFO : gram9-plural-verbs: 46.8% (407/870)
2021-10-30 19:11:04,678 : INFO : Quadruplets with out-of-vocabul

Doc2Vec(dm/c,d100,n5,w5,mc2,t4): 17.54%% correct (2388 of 13617


Even though this is a tiny, domain-specific dataset, it shows some meager
capability on the general word analogies – at least for the DM/mean and
DM/concat models which actually train word vectors. (The untrained
random-initialized words of the DBOW model of course fail miserably.)


